In [2]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

import pandas as pd
from functions import join_stocks_crypto, generate_rand_portfolios, sharpe_ratio_calculation, select_top_five, run_clustering_model
from functions_post_clustering import reoptimize_weights, select_complementary_cryptos, supplement_set_with_cryptos

import cvxpy as cp
import random
import json
import warnings

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [3]:
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')
joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left')

cryptos_list = list(cryptos_df.columns)

In [4]:
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)


#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [5]:
#Optimize Traditional Portfolios
# for i in range(999, 1001):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets, min_weight_for_top_five=0.05)  #TRY DIFFERENT WEIGHTS FOR top_five
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)




#Reassemble the results of the optimization - jsons
# min_var_portfolios = dict()
# for i in range(1,1000):
#     with open(f'min_variance_portfolio_jsons/my_dict{i}.json') as f:
#         port = json.load(f)
#         min_var_portfolios.update(port)

# with open(f"full_optimized_min_variance.json", "w") as f:
#     json.dump(min_var_portfolios, f, indent=4)

In [6]:
# top_five = top_five_dict['portfolio_0']
# top_five

In [10]:
#Actually Cluster the stocks

warnings.simplefilter(action='ignore', category=FutureWarning) #supress warnings for cleanliness
warnings.simplefilter(action='ignore', category=RuntimeWarning)

joined_df.index = pd.to_datetime(joined_df.index)
joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level
joined_df_3days = joined_df.resample('3D').last()
#AHC
df = joined_df_3days.rolling(window=60, center=True).mean()
labels, tickers_with_labels, _, _ = run_clustering_model(df, n_clus=7, model_name='ahc', linkage='complete', return_mode='geometric', n_init=3)

In [ ]:
# mu = expected_returns.mean_historical_return(joined_df)

# S = risk_models.sample_cov(joined_df)

# top_five = {'CRESY': 0.9624262180841339,
#  '601857.SS': 0.8426712750398243,
#  'PKX': 0.7110552210687626,
#  '0386.HK': 0.6714594474023661,
#  'VERX': 0.6467235554934667}


In [ ]:
non_random = supplement_set_with_cryptos(portfolio_set=random_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels, df_prices=joined_df, n_cryptos=3)

random = supplement_set_with_cryptos(portfolio_set=random_portfolios, cryptos_list=cryptos_list, tickers_with_labels=tickers_with_labels, df_prices=joined_df, n_cryptos=3, random_alloc=True)


{'portfolio_0': ['0386.HK',
  'PKX',
  'COST',
  'SU.PA',
  'DG.PA',
  'VERX',
  'TLK',
  'KT',
  '1658.HK',
  '601857.SS',
  'HDB',
  '7203.T',
  'ADYEN.AS',
  'CSCO',
  'CRESY',
  'UNI-USD',
  'INJ-USD',
  'APE-USD'],
 'portfolio_1': ['HMC',
  'SMMT',
  'AIR.PA',
  '600941.SS',
  '8001.T',
  'COST',
  '2914.T',
  'PRCT',
  '0883.HK',
  '601857.SS',
  'AD.AS',
  'TTD',
  'INGA.AS',
  '6861.T',
  'TTE.PA',
  'UNI-USD',
  'INJ-USD',
  'APE-USD'],
 'portfolio_2': ['ADBE',
  'AXSM',
  '6288.HK',
  'ADYEN.AS',
  'QF9.F',
  'TTE.PA',
  'APPF',
  'SMMT',
  'PQ9.F',
  'KT',
  'HMC',
  '0YSU.L',
  'IBN',
  '0A4J.L',
  'SKFA.F',
  'UNI-USD',
  'INJ-USD',
  'APE-USD'],
 'portfolio_3': ['8001.T',
  'SAF.PA',
  'LIN',
  'MT.AS',
  '601658.SS',
  'SONY',
  '0TDD.L',
  'CIB',
  '601939.SS',
  '4AH1.F',
  '9434.T',
  '9432.T',
  '0A7R.L',
  '6098.T',
  'OLLI',
  'UNI-USD',
  'INJ-USD',
  'APE-USD'],
 'portfolio_4': ['TSLA',
  'LIN',
  '1088.HK',
  'AI.PA',
  '3YU.F',
  'CIB',
  'AIR.PA',
  'KEP',
  '

In [ ]:
reoptimize_weights(df_prices=joined_df, portfolio_set=random_portfolios, how='max_sharpe', min_weight=0.01)

In [ ]:
###ADD A FUNCTION TO HAVE RANDOM ASSIGNMENTS OF 3 CRYPTOS
